In [1]:
import sys
import os
    
sys.path.append(os.getcwd())

# Required information

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Load environment variables from .env
aws_account_id = os.getenv('AWS_ACCOUNT_ID')

aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')
aws_region = os.getenv('AWS_REGION')

s3_bucket_name = os.getenv('S3_BUCKET_NAME')
s3_object_key = os.getenv('S3_OBJECT_KEY')

## Specify the role name and trust policy for the Lambda service
iam_role_name = os.getenv('IAM_ROLE_NAME')

## User-dependent solution variables

In [3]:
# User setup

# Platform
ecr_image_name = "serverless-example"

# Function description
func_description='A test Lambda function'

# API
endpoint = "predict"
method_verb='POST'
stage_name = "test"

## Protocols and performance

In [4]:
# AWS setup
from os import getcwd
from deploy_scripts.utils.misc import get_lambda_usage_constraints, get_trust_policy

cwd=getcwd()

## Specify the path to the JSON file
## NOTE: Update with the actual path to your JSON file
trust_policy_file_path = cwd+"/deploy_scripts"
trust_policy = get_trust_policy(trust_policy_file_path)

## Rate limits: Harsh since this will be public facing
## Quota: Low daily limits for the same reason
usage_constraints_file_path = cwd+"/deploy_scripts"
usage_constraints = get_lambda_usage_constraints(usage_constraints_file_path)

## AWS Lambda execution role 
lambda_policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'


Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Loaded JSON:
{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0}, 'quota': {'limit': 100, 'period': 'DAY'}}


# Clients

In [5]:
import boto3

# Set up the IAM client
iam_client = boto3.client('iam', region_name=aws_region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=aws_region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=aws_region)

# Development steps

1. IAM role image handling;
2. ECR image
3. Lambda function creation;
4. API Gateway

## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 



In [ ]:
from deploy_scripts.utils.iam_utils import try_attach_role_policy

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, iam_role_name, lambda_policy_arn, trust_policy)

role_arn

## ECR image

Run 2 cells below with key stroke _Shift+Enter_ to upload the docker image to ECR.

In [ ]:
from deploy_scripts.utils.ecr_utils import pipe_push_image

pipe_push_image(aws_account_id, aws_region, ecr_image_name)

# Lambda function creation

Run 2 cells below with key stroke _Shift+Enter_ to generate a Lambda Function based on Docker Image

### Create function

In [ ]:
from deploy_scripts.utils.ecr_utils import build_ecr_url
from deploy_scripts.utils.lambda_utils import delete_function, create_function

# Function name (not public facing)
lambda_function_name = f'lambda-fn-{ecr_image_name}'

# Retrieve (if already exists) or create a new Lambda function
routed_url = build_ecr_url(aws_account_id, aws_region, ecr_image_name)
deletion_response = delete_function(lambda_client, lambda_function_name)
create_response = create_function(lambda_client, lambda_function_name, func_description, routed_url, role_arn)

create_response

### Get function

In [ ]:
from deploy_scripts.utils.lambda_utils import get_function

get_response = get_function(lambda_client, lambda_function_name)

get_response

### Test lambda function

In [ ]:
from json import dumps, loads

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=lambda_function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = loads(response['Payload'].read())

print(result)

## API Gateway setup

Run the cells below to set the Lambda Function as an Endpoint on API Gateway.

In [ ]:
from deploy_scripts.utils.api_gateway_utils import delete_apis_by_name, has_api

delete_apis_by_name(gateway_client, rest_api_name)

In [ ]:
from deploy_scripts.utils.api_gateway_utils import deploy_rest_api, build_api_url

# Define the name of the API (not public facing)
rest_api_name = lambda_function_name + '-api'

deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    aws_account_id, aws_region, \
    lambda_function_name, rest_api_name, \
    endpoint, method_verb, \
    usage_constraints, stage \
)

deployment_reponse

In [ ]:
api_key=deployment_reponse['api_key']
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, aws_region, endpoint, stage)

print(api_key)
print(api_url)


### Test API Gateway endpoint

In [ ]:
import json
import requests

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)
resp.json()


# Deployment pipeline

In [ ]:
deployment_response = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    json.dump(deployment_response, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

In [6]:
from time import time

from deploy_scripts.utils.iam_utils import try_attach_role_policy

from deploy_scripts.utils.ecr_utils import pipe_push_image, \
    build_ecr_url

from deploy_scripts.utils.lambda_utils import delete_lambda_function, \
    create_lambda_function, \
    get_lambda_function, \
    wait_for_lambda_deployment, \
    get_lambda_function_name, \
    deploy_lambda_function

from deploy_scripts.utils.api_gateway_utils import delete_apis_by_name, \
    has_api, \
    deploy_rest_api, \
    build_api_url, \
    wait_for_api_endpoint

start_time=time()

pipe_push_image(aws_account_id, aws_region, ecr_image_name)

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, iam_role_name, trust_policy)

# Deletes existent lambda function
# FIX: This is too harsh. Try update it!
lambda_function_name = get_lambda_function_name(ecr_image_name)
delete_lambda_function(lambda_client, lambda_function_name)

# Deploys lambda function of ECR image
deploy_lambda_function(
        lambda_client, \
        func_description, \
        aws_account_id, \
        aws_region, \
        ecr_image_name, 
        role_arn)

# Defines the name of the API (not public facing)
rest_api_name = lambda_function_name + '-api'

# Delete previous API Gateway
# FIX: This is too harsh. Try update it! 
delete_apis_by_name(gateway_client, rest_api_name)

# Deploys lambda function as API Gateway endpoint
api_deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    aws_account_id, aws_region, \
    lambda_function_name, rest_api_name, \
    endpoint, method_verb, \
    usage_constraints, stage_name \
)

final_time=time()
duration=final_time-start_time

print(f'Deployment duration is {duration} seconds')

----------------------------------------------------------------------
Task "Docker image upload on AWS ECR" just started!
----------------------------------------------------------------------
Logging in on ECR account...
Deleting existent ECR image...
Creating ECR image...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
----------------------------------------------------------------------
Task "Docker image upload on AWS ECR" finished successfully!
----------------------------------------------------------------------

----------------------------------------------------------------------
Task "Role policy attachment" just started!
----------------------------------------------------------------------
----------------------------------------------------------------------
Task "Role policy attachment" finished successfully!
----------------------------------------------------------------------

----------------------------------------------------------

In [9]:
from json import dump

# Retrieve information from 
rest_api_id=api_deployment_reponse['rest_api_id']
api_key=api_deployment_reponse['api_key']
rest_api_id=api_deployment_reponse['rest_api_id']    

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, aws_region, endpoint, stage_name)

deployment_response = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    dump(deployment_response, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

Deployment information saved to deployment_info.json
